1. Parsing part
2. Data processing part
3. Jaccard similarity for tweets classification
4. Word frequency analysis
5. Tweets clustering

In [ ]:
#Some of the packages can be helpful in this project
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import os
import tweepy as tw
import pandas as pd
from urllib.request import urlopen
import requests
import json
import csv
import datetime
import dateutil.parser
import unicodedata
import time

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import re
import spacy
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import RegexpTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
import re
import en_core_web_sm
import plotly.express as px

from sklearn.cluster import KMeans
from wordcloud import WordCloud
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import nltk
import numpy as np
import re
import pandas as pd 
import pylab as pl
import matplotlib.pyplot as plt

from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import pyplot

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# 1 Parsing part

In [ ]:
file = "5_twitterBBC.csv"
f = open(file, "w")
Headers = "tweet_user, tweet_text,  replies,  retweets\n"
f.write(Headers)
url = "https://twitter.com/BBCWorld"
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

In [ ]:
# Gets the tweet
tweets = soup.find_all("li", attrs={"class":"js-stream-item"})

# Writes tweet fetched in file
for tweet in tweets:
    try:
        if tweet.find('p',{"class":'tweet-text'}):
            tweet_user = tweet.find('span',{"class":'username'}).text.strip()
            tweet_text = tweet.find('p',{"class":'tweet-text'}).text.encode('utf8').strip()
            replies = tweet.find('span',{"class":"ProfileTweet-actionCount"}).text.strip()
            retweets = tweet.find('span', {"class" : "ProfileTweet-action--retweet"}).text.strip()
            # String interpolation technique
            f.write(f'{tweet_user},/^{tweet_text}$/,{replies},{retweets}\n')
    except: AttributeError
f.close()


In [ ]:
consumer_key= '...'
consumer_secret= '...'
auth = tw.OAuthHandler(consumer_key, consumer_secret)
api = tw.API(auth, wait_on_rate_limit=True)
auth = tw.OAuthHandler(consumer_key, consumer_secret)
api = tw.API(auth)
os.environ['TOKEN'] = '...'

In [ ]:
def auth():
    return os.getenv('TOKEN')
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent"

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "xbox lang:rus"
max_results = 100
start_time = "2022-03-23T00:00:00.000Z"
key_word = 'Россия' #Russia in Russian

def current_time(end):
    #Inputs for the request
    end_time = end
    return (end_time)

def create_df(end):
    list_of_lists = []
    json_response = json(end)
    for i in range(0,len(json_response['data'])):
        list_of_lists.append([json_response['data'][i]['created_at'], json_response['data'][i]['public_metrics'].get('retweet_count'), json_response['data'][i]['public_metrics'].get('reply_count'), json_response['data'][i]['public_metrics'].get('like_count'), json_response['data'][i]['public_metrics'].get('quote_count'), json_response['data'][i]['text']])
    
    df_temporary = pd.DataFrame(list_of_lists, columns=['created_at', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'text'])
    end = df_temporary['created_at'].iloc[-1]
    return df_temporary, end;

In [ ]:
end = "2022-03-24T23:59:00.000Z"
df1, end = create_df(end)
q=0
dataframes_list = []

while(q<500):
    df2, end = create_df(end)
    dataframes_list.append(df2)
    q+=1
    
df_ = pd.concat(dataframes_list)

# 2 Data processing part

In [ ]:
datasettime = []
for i in range(0,len(df_)):
    orig_time=df_['created_at'].iloc[i][:-5]
    d = datetime.datetime.strptime(orig_time, '%Y-%m-%dT%H:%M:%S')
    datasettime.append(d)
    
datasettime = pd.DataFrame(datasettime)
datasettime.rename(
    columns={0:"time_stamp"}
          ,inplace=True)

In [ ]:
datasettime

In [ ]:
result = pd.concat([df_, datasettime], axis=1)
result = result.drop(['created_at'], axis = 1)

In [ ]:
# remove the hashtags, mentions and unwanted characters.
def clean_text(result, text):
    result[text]=result[text].apply(str)
    result[text] = result[text].str.lower()
    result[text] = result[text].apply(lambda elem: re.sub(r" rt : @[^\s]+|@[^\s]+|@[A-Za-z0–9]+|@[0–9]|(@[A-Za-z0–9]+)|\n|rt|(@[A-Za-z0–9]+)|http.+|://+", "",   elem)) 
    return result

result = clean_text(result, "text")
result

In [ ]:
result.index = result['time_stamp']
result['tweet'] = result['text']
result = result.drop(['text'], axis = 1)
result = result.drop(['created_at'], axis = 1)

In [ ]:
result = result.dropna()

In [ ]:
result

In [ ]:
df_=result

In [ ]:
#I used here functons from https://github.com/ada-k/TweetsClassification/blob/master/analysis.ipynb
w_tokenizer = WhitespaceTokenizer()
nlp = en_core_web_sm.load() 
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)


def furnished(text):
    final_text = []
    for i in w_tokenizer.tokenize(text):
       if i.lower() not in stop:
           word = lemmatizer.lemmatize(i)
           final_text.append(word.lower())
    return " ".join(final_text)

# 3  Jaccard similarity for tweets classification

I made a premise that there are 3 potential types of comments: negative, neutral and positive

After analyzing some of the tweets, I identified a few words-markers of the author's mood. I will not specify the words I used because of ethical reasons

For this part I used Jaccard similarity for tweets classification. This is the resource I relied on https://medium.com/swlh/tweets-classification-and-clustering-in-python-b107be1ba7c7

In [ ]:
negative_related_words = '''...'''
positive_related_words = '''...'''
neutral_related_words = '''...'''

negative = furnished(negative_related_words)
positive = furnished(positive_related_words)
neutral = furnished(neutral_related_words)

df_.text = df_.text.apply(furnished)

s = negative
words = s.split()
negatives = " ".join(sorted(set(words), key=words.index))

s = positive
words = s.split()
positive = " ".join(sorted(set(words), key=words.index))

s = neutral
words = s.split()
neutral = " ".join(sorted(set(words), key=words.index))
neutral

def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = TfidfVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)
def get_scores(group,tweets):
    scores = []
    for tweet in tweets:
        s = jaccard_similarity(group, tweet)
        scores.append(s)
    return scores

negative_scores = get_scores(negative, df_.text.to_list())
positive_scores = get_scores(positive, df_.text.to_list())
neutral_scores = get_scores(neutral, df_.text.to_list())

negative_scores = pd.DataFrame(negative_scores, columns=['Negative'])
positive_scores = pd.DataFrame(positive_scores, columns=['Positive'])
neutral_scores = pd.DataFrame(neutral_scores, columns=['Neutral'])

frames = pd.concat([negative_scores, positive_scores, neutral_scores], axis=1)

df_ = pd.concat([df_, frames], axis=1)

df_without_text = df_.drop('text', axis = 1)

In [ ]:
df_without_text

In [ ]:
scores_df = pd.concat([pd.DataFrame(positive_scores), pd.DataFrame(neutral_scores), pd.DataFrame(negative_scores)], axis=1)

In [ ]:
scores_df

In [ ]:
def get_classes(l1, l2, l3):
    pos = []
    neu = []
    neg = []

    for i, j, k in zip(l1, l2, l3):
        m = max(i, j, k)
        if m == i:
            pos.append(1)
        else:
            pos.append(0)
        if m == j:
            neu.append(1)
        else:
            neu.append(0)        
        if m == k:
            neg.append(1)
        else:
            neg.append(0)   
            
    return pos, neu, neg
l1 = scores_df.Positive.to_list()
l2 = scores_df.Neutral.to_list()
l3 = scores_df.Negative.to_list()

pos, neu, neg = get_classes(l1, l2, l3)
data = {'pos':pos, 'neu':neu, 'neg':neg}
class_df = pd.DataFrame(data)
new_groups_df = class_df.sum()
new_groups_df['total'] = new_groups_df['pos'] + new_groups_df['neu'] + new_groups_df['neg']

In [ ]:
new_groups_df['total'] = new_groups_df['pos'] + new_groups_df['neu'] + new_groups_df['neg']

This is the result of tweets classification using Jaccard similarity

In [ ]:
new_groups_df

In [ ]:
labels = 'Positive', 'Neutral', 'Negative'
sizes = [new_groups_df.pos, new_groups_df.neu, new_groups_df.neg]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')

plt.show()

In [ ]:
labels = ['Positive','Neutral','Negative']
values = [new_groups_df.pos, new_groups_df.neu, new_groups_df.neg]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial', hole=.3
                            )])
fig.show()

# 4 Word frequency analysis

Now I would like to analyze the frequency of certain words

In [ ]:
words = []
words_all_list = []
w = []

for i in range(0,len(result)):
    words.append(result["text"].iloc[i])
    
for i in range(0,len(words)):
    words_all_list.append(words[i].split(" "))

for i in range(0,len(words_all_list)):
    for j in range(0,len(words_all_list[i])):
        w.append(words_all_list[i][j])

In [ ]:
w

In [ ]:
x = w
unique, counts = np.unique(x, return_counts=True)

a = pd.DataFrame((unique, counts)).T
a = a.sort_values(by=1, ascending=False)
a = a.rename(columns={0: "Word", 1: "Freq"})
a = a.reset_index().drop('index', axis=1)

In [ ]:
a

The larger the word size, the more common it is.

In [ ]:
d = {w: f for w, f in
     zip(np.array(a["Word"][21:500]),
         np.array(a["Freq"][21:500]))}
wordcloud = WordCloud(width=1600, height=800,max_font_size=200, background_color='black', prefer_horizontal=1)
wordcloud.generate_from_frequencies(frequencies=d)
plt.figure(figsize=(18,15))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
mask3 = np.array(Image.open('file.jpeg'))

image_colors = ImageColorGenerator(mask3)

d = {w: f for w, f in
     zip(np.array(a["Word"][21:1000]),
         np.array(a["Freq"][21:1000]))}
wordcloud = WordCloud(width=1600, height=800,max_font_size=200, random_state=100, max_words=1000, background_color='white', prefer_horizontal=1, mask=mask3)
wordcloud.generate_from_frequencies(frequencies=d)
plt.figure(figsize=(180,15))
plt.tight_layout(pad=0)
plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
class_df = pd.DataFrame(data)
# Considering 3 grams and mimnimum frq as 0
tf_idf_vect = CountVectorizer(analyzer='word',ngram_range=(1,1), min_df = 0.0001)
tf_idf_vect.fit(result['text'])
desc_matrix = tf_idf_vect.transform(result["text"])

# 5 Tweets clustering

Now I use the kMeans method for data clustering (unsupervised method)

In [ ]:
num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(desc_matrix)
clusters = km.labels_.tolist()
tweets = {'Tweet': result["text"].tolist(), 'Cluster': clusters}
frame = pd.DataFrame(tweets, index = [clusters])

In [ ]:
frame['Cluster'].value_counts()

I found common patterns within each class

In [ ]:
labels = ['Furious and obscene', 'Negative restrained','Sarcastic, skeptical and meme']
values = frame['Cluster'].value_counts()[0], frame['Cluster'].value_counts()[1], frame['Cluster'].value_counts()[2]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')

plt.show()

In [ ]:
labels = ['Negative restrained','Sarcastic, skeptical and meme','Furious and obscene']
values = frame['Cluster'].value_counts()

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial', hole=.3
                            )])
fig.show()